# Getting Seasonal Means for the season December-January-February

We've been getting occasionally a few questions about seasonal reduction operations (for example means).

Say you have a dataset that goes on for several years, and you want to compare the seaonal means of each year against each other.

For three of the four seasons this isn't a big deal, you cut the dataset into yearly chunks, then mean over the season.

But for the fourth season (DJF) this doesn't work: It bunches together the January, February, and December of that same year, when in fact you want the December from the year before.

Let's have a look at an overly simplified example.

## Our example dataset

We only use a single dimension of data, and we only use monthly values. 
What's more, the values of our "dataset" are strictly ascending. 

This keeps the dataset small and we can easily see what's going on.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
time = xr.DataArray(
    pd.date_range(start='2000-01-01', end='2004-12-31', freq='MS'), 
    dims=('time',)
)

data = xr.DataArray(
    np.arange(len(time)), 
    coords={'time': time},
)
data

<xarray.DataArray (time: 60)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2004-12-01

## Initial idea: groupby
The `.groupby` method can combine values according to a specific option, and the season is one of these. Let's try

In [3]:
data.groupby(data.time.dt.season).mean(dim='time')

<xarray.DataArray (season: 4)>
array([28., 30., 27., 33.])
Coordinates:
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'

Now this isn't what we wanted.
It has basically averaged over all Decembers, Januarys, and Februaries and stored this as a single value for "DJF", and the same for "MAM", "JJA", and "SON".

Note that since this is a one-dimensional array, we could have omitted the `dim='time'` option from the call to `mean()` but you will probably also have other dimensions, and this way we can guarantee that it will only collapse the time dimension.

## Next idea: Resample
The `.resample` method changes the frequency of the dataset, and allows for reduction methods to be applied.

In [4]:
data.resample(time='QS').mean(dim='time')

<xarray.DataArray (time: 20)>
array([ 1.,  4.,  7., 10., 13., 16., 19., 22., 25., 28., 31., 34., 37.,
       40., 43., 46., 49., 52., 55., 58.])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-04-01 ... 2004-10-01

We are now much closer. 
We have quartly means, except that the values don't correspond to seasons.
The solution are Panda's **Anchored Offsets**

Basically we can move the start and end of a period around a bit.

You can read more about it [here](https://pandas.pydata.org/docs/user_guide/timeseries.html), but suffice to say `QS-DEC` does what we want:

In [5]:
season_means = data.resample(time='QS-DEC').mean(dim='time')
season_means

<xarray.DataArray (time: 21)>
array([ 0.5,  3. ,  6. ,  9. , 12. , 15. , 18. , 21. , 24. , 27. , 30. ,
       33. , 36. , 39. , 42. , 45. , 48. , 51. , 54. , 57. , 59. ])
Coordinates:
  * time     (time) datetime64[ns] 1999-12-01 2000-03-01 ... 2004-12-01

## Extracting the season we're interested in.

You might notice that I've already assigned the seasonal means to a new variable. 
I did this because in order to extract just the **DJF** season we're interested in, I need the condensed time dimension:

In [6]:
season_means.sel(time=season_means.time.dt.season=='DJF')

<xarray.DataArray (time: 6)>
array([ 0.5, 12. , 24. , 36. , 48. , 59. ])
Coordinates:
  * time     (time) datetime64[ns] 1999-12-01 2000-12-01 ... 2004-12-01

This is the solution. 6 Values, starting with the mean of Jan/Feb 2000, then the mean of Dec 2000 and Jan and Feb 2001, and so on, until the final value is just the mean of Dec 2004.